# LangChain Expression Language

### What is LCEL?

* A declarative way to build AI pipelines in LangChain.
* Instead of long Python code, you connect components in a chain-like style.

---

### Why LCEL?

* Shorter and cleaner code.
* Easier to read and maintain.
* Works the same for synchronous, asynchronous, batch, and streaming execution.
* Good for debugging and reusing components.

---

### Basic Example

Without LCEL:

```python
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)
result = chain.run("cats")
```

With LCEL:

```python
chain = prompt | ChatOpenAI() | StrOutputParser()
result = chain.invoke({"topic": "cats"})
```
**prompt, ChatOpenAI(), StrOutputParser() are called runnables**

---

## LCEL Operators

* `|` → Pipe operator, connects steps.
* `.invoke()` → Run once with input.
* `.batch()` → Run for multiple inputs.
* `.stream()` → Get outputs step by step (streaming responses).

---

## Key Features

1. Composable – you can plug and play different parts.
2. Parallel and sequential execution supported.
3. Unified API – works the same for sync, async, batch, and streaming.
4. Production-ready – optimized for speed and scalability.

---

## Simple Summary

* LCEL is a short and readable way to connect LangChain components.
* It works like piping data through different steps.
* Use `|` to chain steps.
* The same code works for real-time, async, or batch calls.

---


## Piping a prompt, model, and an output parser

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
import config

In [3]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [16]:
# Chat Template

chat_template = ChatPromptTemplate.from_messages([
    ('human', "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)
])

print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [17]:
# Chat Model

def AskGeminiAI(question):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )
    response = llm.invoke(question)
    return response.content

In [18]:
# Output parser

list_output_parser = CommaSeparatedListOutputParser()

In [28]:
# 3 step

chat_template_result = chat_template.invoke({'pet':'dog'})

chat_result = AskGeminiAI(chat_template_result)

list_output_parser.invoke(chat_result)

['Buddy', 'Luna', 'Charlie']

In [29]:
# Single line

chain = chat_template | AskGeminiAI | list_output_parser

chain.invoke({'pet':'dog'})

['Buddy', 'Luna', 'Charlie']

## Batching

In [30]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', 
     "I've recently adopted a {pet} which is a {breed}. Could you suggest several training tips?")
])

In [31]:
chain = chat_template | AskGeminiAI

In [32]:
chain.invoke({'pet':'dog', 'breed':'shepherd'})

'Congratulations on adopting a Shepherd! They are intelligent and loyal dogs, but can also be energetic and require consistent training. Here are some training tips tailored to Shepherds:\n\n**1. Establish Yourself as the Leader (Positive Reinforcement Focus):**\n\n*   **Consistency is Key:** Shepherds thrive on structure and routine. Be consistent with your commands, rules, and expectations. Everyone in the household should use the same commands and enforce the same rules.\n*   **Positive Reinforcement:** Shepherds respond very well to positive reinforcement. Use treats, praise, toys, and affection to reward desired behaviors.  Avoid punishment-based methods, as they can damage your relationship and lead to anxiety or aggression.\n*   **Clear Communication:** Use clear and concise commands. Avoid long sentences or confusing jargon. Keep your tone consistent and confident.\n\n**2. Basic Obedience Training:**\n\n*   **Start Early:** Begin training as soon as possible, even if your Sheph

In [37]:
%%time
chain.batch([{'pet':'dog', 'breed':'shepherd'}, 
             {'pet':'dragon', 'breed':'night fury'}])

CPU times: total: 109 ms
Wall time: 3.03 s


["Congratulations on adopting a Shepherd! They are intelligent and eager to please, but also need consistent training and mental stimulation. Here are some training tips specifically geared towards Shepherds:\n\n**1. Early Socialization is Key:**\n\n*   **Expose your Shepherd to a variety of sights, sounds, people, and other dogs.**  Do this in a controlled and positive manner.  Think puppy classes, walks in different environments, and meeting friendly, well-behaved dogs.\n*   **Socialization window:**  The critical socialization period is between 8 and 16 weeks of age.  While it's ideal to start during this time, socialization is important throughout their life.\n*   **Positive Reinforcement:**  Use treats, praise, and toys to reward positive interactions.  Never force an interaction if your dog is scared or uncomfortable.\n\n**2. Basic Obedience is Essential:**\n\n*   **Start with the basics:**  Sit, stay, come, down, leave it, and heel.  Use positive reinforcement methods.\n*   **Co

In [38]:
%%time
chain.invoke({'pet':'dog', 'breed':'shepherd'})

CPU times: total: 62.5 ms
Wall time: 2.63 s


'Congratulations on adopting a shepherd! They\'re intelligent and eager to please, but also need consistent training and mental stimulation. Here are some training tips specific to shepherds:\n\n**1. Focus on Positive Reinforcement:**\n\n*   **Treats, Praise, and Toys:** Shepherds respond very well to positive reinforcement. Use high-value treats (small, smelly, and irresistible!), enthusiastic praise, and favorite toys as rewards for good behavior.\n*   **Avoid Punishment:** Punishment can damage your bond and lead to fear-based aggression. Focus on rewarding the behaviors you want to see.\n*   **Clicker Training:** Consider using a clicker. It\'s a great way to mark the exact moment your dog performs the desired behavior, followed by a reward.\n\n**2. Start Early and Be Consistent:**\n\n*   **Basic Obedience:** Teach basic commands like "sit," "stay," "down," "come," and "leave it." Start in a quiet environment and gradually introduce distractions.\n*   **Consistency is Key:** Everyo

In [39]:
%%time
chain.invoke({'pet':'dragon', 'breed':'night fury'})

CPU times: total: 78.1 ms
Wall time: 3.86 s


"Congratulations on adopting a Night Fury! They are incredibly intelligent, loyal, and powerful dragons, but also require a special approach to training. Here are some training tips, keeping in mind their unique personality:\n\n**Building Trust and Connection (Crucial for Night Furies):**\n\n*   **Patience and Understanding:** This is paramount. Night Furies are naturally wary and independent. Rushing the process will only push them away. Go slow, be gentle, and respect their boundaries.\n*   **Positive Reinforcement:** Absolutely essential. Forget punishment. Focus on rewarding desired behaviors with praise, affection (scratches under the chin are often appreciated!), and treats (more on that later).\n*   **Non-Verbal Communication:** Pay close attention to your Night Fury's body language. Learn to read their ear positions, tail movements, and vocalizations. They are excellent communicators, but you need to listen.\n*   **Bonding Time:** Spend quality time with your Night Fury without

### 1. **CPU time**

* The amount of time the CPU actually spent executing your code.
* Includes user + system CPU usage.
* If your code runs in **parallel** across multiple cores, CPU time can be **greater than wall time** (because it sums across cores).

---

### 2. **Wall time**

* The real-world elapsed time (like looking at a stopwatch).
* Starts when the cell begins execution, stops when it ends.
* Includes waiting, I/O delays, and any time the process was idle.

---